# Validating Centroid Data

This is a short notebook for you to sanity-check that the centroid data looks correct, i.e., that the distance from the bat to each microphone matches the order in which the chirps reach the microphones.

If you haven't gone throug the **Step 0: Prerequisites** part of `Walkthrough.ipynb` notebook, please do so before running this notebook.

In [ ]:
import Pkg;
Pkg.develop(path="../BatlabJuliaUtils");
using BatlabJuliaUtils;
using Plots;
using Printf;
using MAT;
using Statistics;
using Roots;
using DataInterpolations;
using DSP;
using WAV;

PLOT_COLORS = [27, 209, 34, 170, 136, 30, 204, 172];

## Specify MAT Files

In [ ]:
AUDIO_FILENAME = "../data/Pu166_02.mat";
CENTROID_FILENAME = "../data/centroid/Pu166_002_centroidxyz.mat";
MIC_POSITION_FILENAME = "../data/mic_positions_fall2021.mat";

CENTROID_VARIABLE_NAME = collect(keys(matread(CENTROID_FILENAME)))[1];
MIC_POSITION_VARIABLE_NAME = collect(keys(matread(MIC_POSITION_FILENAME)))[1];

In [ ]:
(@printf "Do the variable names for these MAT files look right?\n\tFor the centroid file: \"%s\",\n\tand for the mic position file: \"%s\"" CENTROID_VARIABLE_NAME MIC_POSITION_VARIABLE_NAME)

If the variable names for the MAT files don't look right, then uncomment and run the following two cells.

In [ ]:
# println("The keys of the centroid file are:\n", keys(matread(CENTROID_FILENAME)), "\nand the MAT file looks like")
# centroids = matread(CENTROID_FILENAME)

In [ ]:
# println("The keys of the mic position file are:\n", keys(matread(MIC_POSITION_FILENAME)), "\nand the MAT file looks like")
# mic_locations = matread(MIC_POSITION_FILENAME)

## Read Audio and Centroid Data

In [ ]:
y = readmicdata(AUDIO_FILENAME);

In [ ]:
centroids = matread(CENTROID_FILENAME)[CENTROID_VARIABLE_NAME];
if size(centroids, 1) == 3
    centroids = Matrix(transpose(centroids));
end
centroids[end-9:end, :]

In [ ]:
mic_positions = matread(MIC_POSITION_FILENAME)[MIC_POSITION_VARIABLE_NAME];
if size(mic_positions, 1) == 3
    mic_positions = Matrix(transpose(mic_positions));
end
mic_positions

In [ ]:
first_non_nan_idx = findfirst(.~isnan.(centroids[:, 1]));
first_non_nan_time = 1000*videoindextosec(first_non_nan_idx, size(centroids, 1));
(@printf "First time where the centroid data is not NaN: %d milliseconds\n" first_non_nan_time)

## Get the distance of the bat to each microphone

The distance in millimeters of the bat to each microphone, for each index of the centroid data, is stored in `distance_to_mics`. Each column represents a different microphone.

In [ ]:
n_mics = size(mic_positions, 1);
distance_to_mics = zeros(size(centroids, 1), n_mics);
for i=1:n_mics
    distance_to_mics[:, i] = distancefrommic(centroids, mic_positions, i);
end

## Which microphone should the chirps reach first?

The following function determines the order in which the each microphone should hear the vocalizations (according to `distance_to_mics`).

Note that the times are in milliseconds since the beginning of the audio data we just read in.

In [ ]:
function getexpectedchirporder(distance_to_mics::Matrix, start_millis::Real, stop_millis::Real)
    L_video = size(distance_to_mics, 1);
    video_idxs = sectovideoindex(start_millis/1000, L_video):sectovideoindex(stop_millis/1000, L_video);
    n_mics = size(distance_to_mics, 2);

    section_start_idx=video_idxs[1];
    ordering = sortperm(distance_to_mics[section_start_idx, :]);
    for i=video_idxs[2:end]
        new_ordering = sortperm(distance_to_mics[i, :]);
        if new_ordering != ordering 
            @printf "For vocalizations from %d to %d millis, the chirp should reach the microphones in the order: " round(videoindextosec(section_start_idx, L_video)*1000) round(videoindextosec(i-1, L_video)*1000)
            [printstyled(string(ordering[k]) * " ", color=PLOT_COLORS[ordering[k]], bold=true) for k=1:n_mics]
            println();
            section_start_idx = i;
        end
        ordering = new_ordering;
    end
    @printf "For vocalizations from %d to %d millis, the chirp should reach the microphones in the order: " round(videoindextosec(section_start_idx, L_video)*1000) stop_millis
    [printstyled(string(ordering[k]) * " ", color=PLOT_COLORS[ordering[k]], bold=true) for k=1:n_mics]
    println();
end

In [ ]:
getexpectedchirporder(distance_to_mics, first_non_nan_time, 8000)

You can use `plotmicdata` on any time segment to verify whether this seems reasonable.

In [ ]:
## CHANGE START_MILLIS AND STOP_MILLIS TO THE TIMES YOU WANT
START_MILLIS = first_non_nan_time;
STOP_MILLIS = first_non_nan_time+300;
############################################################

plotmicdata(Int64(round(START_MILLIS/1000*FS)):Int64(round(STOP_MILLIS/1000*FS)), y, size=(1500, 300))

## Plot the microphone data alongside the distances

The follow function takes in a time range (in milliseconds since the beginning of the audio data stored in `y`) and plots the microphone data directly underneath the distance from each microphone.
Here, you can visually verify that the chirp reaches the closer microphones first.

In [ ]:
function plotdistanceandoscillogram(distance_to_mics::Matrix, mic_data::Matrix, start_millis::Real, stop_millis::Real)
    L_video = size(distance_to_mics, 1);
    video_idxs = sectovideoindex(start_millis/1000, L_video):sectovideoindex(stop_millis/1000, L_video);
    audio_idxs = Int(round(start_millis/1000*FS)):Int(round(stop_millis/1000*FS));

    video_plot_millis = 1000 .* videoindextosec.(video_idxs, L_video);
    p1 = myplot(video_plot_millis, distance_to_mics[video_idxs, :], xlabel="Milliseconds", ylabel="Distance (millimeters)", title="Distance from each microphone", linewidth=1.5, yrange=(0, 6000));
    p2 = plotmicdata(audio_idxs, mic_data)
    return plot(p1, p2, layout=(2, 1), size=(1500, 500))
end

In [ ]:
## CHANGE START_MILLIS AND STOP_MILLIS TO THE TIMES YOU WANT
START_MILLIS = first_non_nan_time;
STOP_MILLIS = first_non_nan_time+300;
############################################################

plotdistanceandoscillogram(distance_to_mics, y, START_MILLIS, STOP_MILLIS)